In [ ]:
pip install pandas

The following code is to obtain the Hibor data from two sources, namely:
1. HKMA
2. Hang Seng Bank


In [ ]:
url1="https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-daily?segment=hibor.fixing"
url2="https://rbwm-api.hsbc.com.hk/pws-hk-hase-rates-papi-prod-proxy/v1/historical-onemonth-hibor?year=##YYYY##&month=##MM##"

date = []
hibor = []
'''
#Previous Data
import urllib.request as req, json
res = req.urlopen(url1)
rec = json.loads(res.read())
rec = rec['result']['records']
for r in rec:
  date.append(r['end_of_day']) 
  hibor.append(r['ir_1m'])
'''
#current month
import datetime
import urllib.request as req, json
year = datetime.datetime.now().year
month = datetime.datetime.now().month
for y in range(2018,2022+1):
    for m in range(1,12+1):
        if not (year==y and month<m):
            url2_u = url2.replace('##YYYY##',str(y))
            url2_u = url2_u.replace('##MM##',str(m).zfill(2))
            res = req.urlopen(url2_u)
            rec = json.loads(res.read())
            rec = rec['historicalOneMonthHibor']
            for r in rec:
                date.append(r['rateDate'])
                hibor.append(r['haseHibor'])

import pandas as pd
df = {'Hibor':hibor,'Date':date}
df = pd.DataFrame(df)
df['Date']=pd.to_datetime(df['Date'])
df['Hibor']=pd.to_numeric(df['Hibor'])
df = df.sort_values('Date',ignore_index=True)
#df.to_csv("data.csv")
#from google.colab import files
#files.download("data.csv")

In [ ]:
pip install backtesting

In [ ]:
print(df.to_string())

In [ ]:
url3='https://api.hkma.gov.hk/public/market-data-and-statistics/daily-monetary-statistics/daily-figures-monetary-base?pagesize=1000'
res = req.urlopen(url3)
rec = json.loads(res.read())
rec = rec['result']['records']
eod = []
balance = []
for r in rec:
  eod.append(r['end_of_date'])
  balance.append(r['aggr_balance_af_disc_win'])
eod, balance
df1 = {'Balance':balance,'Date':eod}
df1 = pd.DataFrame(df1)
df1['Date']=pd.to_datetime(df1['Date'])
df1['Balance']=pd.to_numeric(df1['Balance'])
df1 = df1.sort_values('Date',ignore_index=True)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
fig,ax = plt.subplots()

ax.plot(df['Date'],df['Hibor'],label='Hibor',color='black')
ax1 = ax.twinx()
ax1.plot(df1['Date'],df1['Balance'],label='Balance',color='orange')

import matplotlib.dates as mdates
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b-%y'))

# New section